<b> Import Library, find available devices </b>

In [1]:
# Initialization code for Beta Multiplier Reference Testing.
# This block identifies and initializes the connected SMUs (Source Measure Units).
# If running for the first time, ensure the SMU names are correctly referenced in the subsequent code block.

# This program (ENTER DESCRIPTION)
# This program 
# This program creates a .csv file to save the measurements.

# The setup assumes the use of three SMUs:
# - 
# - 

import pyvisa                    # Importing the pyvisa library for instrument communication
import time                      # Importing the time module to handle time-related tasks (e.g., delays)
import matplotlib.pyplot as plt  # Importing matplotlib.pyplot for plotting graphs and visualizing data
import numpy as np               # Importing numpy for numerical operations, particularly with arrays
import pandas as dp              # Importing pandas as dp for data manipulation and analysis
import os                        # Importing os to interact with the operating system, such as handling file paths
import csv                       # Importing csv to read from and write to CSV files
from datetime import date           # Importing date to read current date
rm = pyvisa.ResourceManager()    # Create a ResourceManager object, which manages communication with instruments
rm.list_resources()              # List all available resources (e.g., connected instruments) managed by pyvisa

('GPIB6::6::INSTR',
 'GPIB6::20::INSTR',
 'GPIB6::21::INSTR',
 'GPIB6::22::INSTR',
 'USB0::0x0957::0x0607::MY45002108::0::INSTR',
 'USB0::0x0957::0x1796::MY57231548::0::INSTR',
 'USB0::0x0957::0x1796::MY57231553::0::INSTR',
 'USB0::0x0957::0x17B6::MY56310504::0::INSTR',
 'USB0::0x0957::0x2C07::MY57890777::0::INSTR',
 'USB0::0x0957::0x5707::MY53802060::0::INSTR',
 'USB0::0x2A8D::0x0101::MY54505624::0::INSTR',
 'USB0::0x2A8D::0x0101::MY57508183::0::INSTR')

In [2]:
#Establishing Connection for NMOS tap SMU
SM_NMOS = rm.open_resource('GPIB6::20::INSTR')  # Open a connection to the instrument with the address 'ASRL3::INSTR' and assign it to SM_VIN
SM_NMOS.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
SM_NMOS.write_termination = '\n'            # Set the write termination character to a newline. This ensures that each command sent to the instrument is properly terminated.
SM_NMOS.baud_rate = 9600                    # Set the baud rate for serial communication to 9600. This is the speed at which data is transmitted over the serial connection.
print(SM_NMOS)                              # Print the resource object (SM_VIN) to display its properties
print(SM_NMOS.query('*IDN?'))               # Send the '*IDN?' command to the instrument and print its response. This command usually asks the instrument to identify itself (e.g., manufacturer, model number).

#Establishing Connection for PMOS tap SMU
SM_PMOS = rm.open_resource('GPIB6::21::INSTR') # Open a connection to the instrument with the address 'ASRL4::INSTR' and assign it to SM_VOUT
SM_PMOS.read_termination = '\n'            # Set the read termination character to a newline for SM_VOUT, similar to SM_VIN
SM_PMOS.write_termination = '\n'           # Set the write termination character to a newline for SM_VOUT, similar to SM_VIN
SM_PMOS.baud_rate = 9600                   # Set the baud rate for SM_VOUT to 9600, ensuring both instruments communicate at the same speed
print(SM_PMOS)                             # Print the resource object (SM_VOUT) to display its properties
print(SM_PMOS.query('*IDN?'))              # Send the '*IDN?' command to the SM_VOUT instrument and print its response, identifying the instrument

#Establishing Connection for VDD SMU
SM_VDD = rm.open_resource('GPIB6::22::INSTR') # Open a connection to the instrument with the address 'ASRL4::INSTR' and assign it to SM_VOUT
SM_VDD.read_termination = '\n'            # Set the read termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VDD.write_termination = '\n'           # Set the write termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VDD.baud_rate = 9600                   # Set the baud rate for SM_VOUT to 9600, ensuring both instruments communicate at the same speed
print(SM_VDD)                             # Print the resource object (SM_VOUT) to display its properties
print(SM_VDD.query('*IDN?'))              # Send the '*IDN?' command to the SM_VOUT instrument and print its response, identifying the instrument


#Establishing Connection for Temperature Chamber
CHAMBER = rm.open_resource('GPIB6::6::INSTR') 
CHAMBER.read_termination = '\n'            
CHAMBER.write_termination = '\n'           
CHAMBER.baud_rate = 9600                  
print(CHAMBER)                            
print(CHAMBER.query('STATUS?'))              

VDD_MAX=1.2
currentRange = .0105

TEMPlist = [125, 100, 75, 50, 25, 0, -25, -50, -75, -100, -125, -150, -175]      # List of temperatures for test

GPIBInstrument at GPIB6::20::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2401,4636506,B02 Jan 20 2021 10:19:49/B01  /W/N
GPIBInstrument at GPIB6::21::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2401,4636504,B02 Jan 20 2021 10:19:49/B01  /W/N
GPIBInstrument at GPIB6::22::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2400,1033554,C27   Feb  4 2004 14:58:04/A02  /K/H
GPIBInstrument at GPIB6::6::INSTR
NNNNNNNNNNNNNNNNNN0


<b> Enter the Device under Test <b>

In [3]:
directory_name = input("Enter the Device under Test (DUT):")  # Prompt the user to input the temperature of the experiment, which will be used as the directory name
DUT="DUT_"+str(directory_name)
os.makedirs(DUT, exist_ok=True)                             # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True
DATE=str(DUT)+"/"+str(date.today())                         # Create another directory for test date within DUT name
os.makedirs(DATE, exist_ok=True) 

<b> Test Sweep Definitions </b>

In [12]:
def Chamber_INIT():
    CHAMBER.write("ON")
    CHAMBER.write("RATE=20")            # Sets the speed at which the chamber will ramp to change TEMP
    CHAMBER.write("WAIT=30")            # Sets the time that the chamber will wait once the desired temperature is reached before measurements start
    CHAMBER.write("HON")                # Turns heat on
    CHAMBER.write("CON")                # Turns cool on
    time.sleep(1) 

    print(CHAMBER.query("RATE?"))
    print(CHAMBER.query("WAIT?"))
    return

def VDD_Sweep():
    # Reset both SMUs to default settings and clear errors
    SM_PMOS.write("*RST; *CLS")
    SM_VDD.write("*RST; *CLS")
    # Clear trace buffers and triggers
    SM_PMOS.write("TRAC:CLE")
    SM_VDD.write("TRAC:CLE")
    SM_PMOS.write("TRIG:CLE")
    SM_VDD.write("TRIG:CLE")

    # Configure SM_PMOS (Master)
    SM_PMOS.timeout = 200000                          # Set timeout for SMU operations
    SM_PMOS.write("SYST:AZER ON")                     # Enable auto-zero
    SM_PMOS.write("SOUR:FUNC VOLT")                   # Set source function to voltage
    SM_PMOS.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
    SM_PMOS.write("SOUR:VOLT:START 0.000")            # Set start voltage
    SM_PMOS.write("SOUR:VOLT:STOP 1.200")             # Set stop voltage
    SM_PMOS.write("SOUR:VOLT:STEP 0.01")             # Set voltage step size
    SM_PMOS.write("SENS:FUNC 'CURR'")                 # Set measurement function to current
    SM_PMOS.write("SENS:CURR:PROT 0.0105")           # Set current protection (compliance) to 50 µA
    SM_PMOS.write("SENS:CURR:RANG " + str(currentRange))            # Set current range
    SM_PMOS.write("SENS:CURR:NPLC 1")                 # Set NPLC for current measurement to 1
    SM_PMOS.write("ARM:COUN 1")
    SM_PMOS.write("ARM:SOUR IMM")
    SM_PMOS.write("ARM:DIR ACC")
    SM_PMOS.write("ARM:OUTP NONE")
    SM_PMOS.write("TRIG:COUN 121")                   # Set trigger count to match sweep points
    SM_PMOS.write("TRIG:SOUR TLIN")                    # Immediate trigger source for the master
    SM_PMOS.write("TRIG:OUTP SOUR")                   # Set trigger output to Trigger Link
    SM_PMOS.write("TRIG:INP SENS")
    SM_PMOS.write("TRIG:DIR ACC")
    SM_PMOS.write("TRIG:OLIN 2")
    SM_PMOS.write("TRIG:ILIN 1")
    SM_PMOS.write("OUTP ON")                          # Enable output

    # Configure SM_VDD (Slave)
    SM_VDD.timeout = 200000                           # Set timeout for SMU operations
    SM_VDD.write("SYST:AZER ON")                      # Enable auto-zero
    SM_VDD.write("SOUR:FUNC VOLT")                    # Set source function to voltage
    SM_VDD.write("SOUR:VOLT:START 0")                # Set start voltage
    SM_VDD.write("SOUR:VOLT:STOP 1.2")                   # Set stop voltage
    SM_VDD.write("SOUR:VOLT:STEP 0.01")             # Set voltage step size
    SM_VDD.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
    SM_VDD.write("SENS:FUNC 'CURR'")                  # Set measurement function to current
    SM_VDD.write("SENS:CURR:PROT 0.0105")            # Set current protection (compliance) to 50 µA
    SM_VDD.write("SENS:CURR:RANG " + str(currentRange))             # Set current range
    SM_VDD.write("SENS:CURR:NPLC 1")                  # Set NPLC for current measurement to 1
    SM_VDD.write("TRIG:COUN 121")                    # Set trigger count to match sweep points
    SM_VDD.write("TRIG:SOUR TLIN")                    # Trigger source is Trigger Link (slave)
    SM_VDD.write("TRIG:INP SOUR")
    SM_VDD.write("TRIG:OUTP SENS")
    SM_VDD.write("TRIG:OLIN 1")
    SM_VDD.write("TRIG:ILIN 2")
    SM_VDD.write("OUTP ON")                           # Enable output

    # Start the measurement
    time.sleep(1)
    SM_VDD.write("INIT")                              # Ensure SM_VDD is also ready
    PMOSdata=SM_PMOS.query("READ?").split(',')                             # Initiate the sweep on SM_PMOS
    VDDdata=SM_VDD.query("FETC?").split(',')
    time.sleep(1)
    PMOSV=PMOSdata[0::5]
    PMOSI=PMOSdata[1::5]
    VDD=VDDdata[0::5]
    IDD=VDDdata[1::5]

    df = dp.DataFrame({'V PMOS':PMOSV, 'I PMOS':PMOSI, 'VDD':VDD, 'IDD':IDD})   # Create a DataFrame
    csv_path=os.path.join(DATE, 'VDD_Sweep.csv')                                             # Define the file path for saving the CSV file in the specified directory.
    df.to_csv(csv_path, index=False)

    return

def NMOS_Sweep():
    SM_NMOS.write("*RST; *CLS")
    #SM_NMOS.timeout = 200000                  # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    # Clear trace buffers and triggers
    SM_NMOS.write("TRAC:CLE")
    SM_NMOS.write("TRIG:CLE")

    SM_NMOS.write(":SOUR:VOLT:STAR 0")
    SM_NMOS.write(":SOUR:VOLT:STOP 1.2")
    SM_NMOS.write(":SOUR:VOLT:STEP .01")

    SM_NMOS.write(":SOUR:VOLT:MODE SWE")
    SM_NMOS.write("SENS:FUNC 'CURR'")
    SM_NMOS.write("SENS:CURR:PROT:LEV 0.0105")
    SM_NMOS.write("SENS:CURR:RANG " + str(currentRange))
    
    #SM_NMOS.write(":SOUR:SWE:SPAC LIN")
    #SM_NMOS.write(":SOUR:SWE:RANG AUTO")      # Set the sweep range to auto, allowing the SMU to automatically adjust the range.
    SM_NMOS.write(":SOUR:SWE:SPAC LIN")       # Set the sweep spacing to linear, meaning the voltage will increase linearly between steps.
    #SM_NMOS.write(":SOUR:SWE:CAB EARL")       # Set the sweep to begin before the trigger event (EARL mode).

    SM_NMOS.write(":TRIG:COUN 121")
    SM_NMOS.write(":SOUR:DEL 0.001")

    SM_NMOS.write("OUTP ON")

    # Start the measurement
    time.sleep(1)
    NMOSdata = SM_NMOS.query(":READ?").split(',')
    time.sleep(1)
    NMOSV=NMOSdata[0::5]
    NMOSI=NMOSdata[1::5]

    df = dp.DataFrame({'V NMOS':NMOSV, 'I NMOS':NMOSI})   # Create a DataFrame
    csv_path=os.path.join(DATE, 'NMOS_Sweep.csv')                                             # Define the file path for saving the CSV file in the specified directory.
    df.to_csv(csv_path, index=False)
    return

def PMOS_Sweep():
    SM_PMOS.write("*RST; *CLS")
    #SM_PMOS.timeout = 200000                  # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    # Clear trace buffers and triggers
    SM_PMOS.write("TRAC:CLE")
    SM_PMOS.write("TRIG:CLE")

    SM_PMOS.write(":SOUR:VOLT:STAR 0")
    SM_PMOS.write(":SOUR:VOLT:STOP 1.2")
    SM_PMOS.write(":SOUR:VOLT:STEP .01")

    SM_PMOS.write(":SOUR:VOLT:MODE SWE")
    SM_PMOS.write("SENS:FUNC 'CURR'")
    SM_PMOS.write("SENS:CURR:PROT:LEV 0.0105")
    SM_PMOS.write("SENS:CURR:RANG " + str(currentRange))
    
    #SM_PMOS.write(":SOUR:SWE:SPAC LIN")
    #SM_PMOS.write(":SOUR:SWE:RANG AUTO")      # Set the sweep range to auto, allowing the SMU to automatically adjust the range.
    SM_PMOS.write(":SOUR:SWE:SPAC LIN")       # Set the sweep spacing to linear, meaning the voltage will increase linearly between steps.
    #SM_PMOS.write(":SOUR:SWE:CAB EARL")       # Set the sweep to begin before the trigger event (EARL mode).

    SM_PMOS.write(":TRIG:COUN 121")
    SM_PMOS.write(":SOUR:DEL 0.001")
    SM_PMOS.write("OUTP ON")

    # Start the measurement
    PMOSdata = SM_PMOS.query(":READ?").split(',')
    PMOSV=PMOSdata[0::5]
    PMOSI=PMOSdata[1::5]

    df = dp.DataFrame({'V PMOS':PMOSV, 'I PMOS':PMOSI})   # Create a DataFrame
    csv_path=os.path.join(DATE, 'PMOS_Sweep.csv')                                             # Define the file path for saving the CSV file in the specified directory.
    df.to_csv(csv_path, index=False)
    return

def VDD_On():
    SM_VDD.write(":SOUR:VOLT:MODE FIX")
    #SM_VDD.write("SOUR:VOLT:LEV 1.2")
    SM_VDD.write(":OUTP ON")
    return

def NMOS_On():
    SM_NMOS.write(":SOUR:VOLT:MODE FIX")
    #SM_NMOS.write("SOUR:VOLT:LEV 0")
    SM_NMOS.write(":OUTP ON")
    return

def PMOS_On():
    SM_PMOS.write(":SOUR:VOLT:MODE FIX")
    #SM_PMOS.write("SOUR:VOLT:LEV 1.2")
    SM_PMOS.write(":OUTP ON")
    return

def VDD_INIT():
    SM_VDD.write(":CONF:CURR")                  # Configure the SMU to measure current
    SM_VDD.write("OUTP:STAT 0")                 # Turn off the VIN source meter
    SM_VDD.timeout = 200000                     # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    SM_VDD.write("SOUR:VOLT:RANG 1.2")
    SM_VDD.write("SENS:CURR:PROT:LEV 0.0105")   # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
    SM_VDD.write("SENS:CURR:RANG " + str(currentRange))         # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
    SM_VDD.write("CURR:NPLC 1")                 # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.
    SM_VDD.write("SOUR:VOLT:LEV 1.2")
    return

def NMOS_INIT():
    SM_NMOS.write(":CONF:CURR")                 # Configure the SMU to measure current
    SM_NMOS.write("OUTP:STAT 0")                # Turn off the VIN source meter
    SM_NMOS.timeout = 200000                    # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    SM_NMOS.write("SOUR:VOLT:RANG 1.2")
    SM_NMOS.write("SENS:CURR:PROT:LEV 0.0105")  # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
    SM_NMOS.write("SENS:CURR:RANG " + str(currentRange))        # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
    SM_NMOS.write("CURR:NPLC 1")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.
    SM_NMOS.write("SOUR:VOLT:LEV 0")
    return

def PMOS_INIT():
    SM_PMOS.write(":CONF:CURR")                 # Configure the SMU to measure current
    SM_PMOS.write("OUTP:STAT 0")                # Turn off the VIN source meter
    SM_PMOS.timeout = 200000                    # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    SM_PMOS.write("SOUR:VOLT:RANG 1.2")
    SM_PMOS.write("SENS:CURR:PROT:LEV 0.0105")  # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
    SM_PMOS.write("SENS:CURR:RANG " + str(currentRange))        # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
    SM_PMOS.write("CURR:NPLC 1")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                                # This controls the integration time, with higher values leading to more accurate but slower measurements.
    SM_PMOS.write("SOUR:VOLT:LEV 1.2")
    return



In [13]:
VDD_INIT()
NMOS_INIT()
PMOS_INIT()

NMOS_On()
VDD_On()
PMOS_On()

VDD_Sweep()
time.sleep(1)
VDD_INIT()
PMOS_INIT()
VDD_On()
PMOS_On()
time.sleep(1)
######################################################
##################### NMOS Sweep #####################
######################################################
NMOS_Sweep()
time.sleep(1)
NMOS_INIT()
NMOS_On()
time.sleep(1)
######################################################
##################### PMOS Sweep #####################
######################################################
PMOS_Sweep()
time.sleep(1)
PMOS_INIT()
PMOS_On()   
time.sleep(1)

<b> Test Protocol </b>

In [ ]:
#TODO: add an initialization section to set ranges so that functions can be condensed
Chamber_INIT()

VDD_INIT()
NMOS_INIT()
PMOS_INIT()

NMOS_On()
VDD_On()
PMOS_On()


for TEMP in TEMPlist:
    local=str(DATE)+"/TEMP_"+str(TEMP)                           # The temperature of the experiment gets set as the sub directory name
    os.makedirs(local, exist_ok=True)                           # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True
    
    CHAMBER.write("WAIT=30")                                    # Resets the wait time of the temp chamber to 30 minutes so it doesn't time out mid test
    CHAMBER.write("SET=" + str(TEMP))
    # TODO: There is probably a better way to check for temperature settling and do wait time
    while (CHAMBER.query("STATUS?")[3] == 'N'):                 # Checks if the temperature chamber has arrived at TEMP yet or not
        time.sleep(15)
        print(CHAMBER.query("STATUS?"))  
    print(CHAMBER.query("STATUS?"))
    time.sleep(300)                                              # Soak time


    #####################################################
    ##################### VDD Sweep #####################
    #####################################################
    VDD_Sweep()
    time.sleep(1)
    VDD_INIT()
    PMOS_INIT()
    VDD_On()
    PMOS_On()
    time.sleep(1)
    ######################################################
    ##################### NMOS Sweep #####################
    ######################################################
    NMOS_Sweep()
    time.sleep(1)
    NMOS_INIT()
    NMOS_On()
    time.sleep(1)
    ######################################################
    ##################### PMOS Sweep #####################
    ######################################################
    PMOS_Sweep()
    time.sleep(1)
    PMOS_INIT()
    PMOS_On()   
    time.sleep(1)
###############################################################
########################## Cold Start #########################
###############################################################
SM_PMOS.write(":OUTP OFF")
SM_NMOS.write(":OUTP OFF")
SM_VDD.write(":OUTP OFF")

CHAMBER.write("WAIT=30")                                        # Reset wait time so that it doesn't time out
time.sleep(900)                                                 # Wait 15 minutes for device to cool down
CHAMBER.write("WAIT=30")                                        # Same as above (TODO: There is probably a better way to do this)
    
NMOS_On()
VDD_On()
PMOS_On()
time.sleep(300)

local=str(DATE)+"/COLD_START"                           # The temperature of the experiment gets set as the sub directory name
os.makedirs(local, exist_ok=True)

VDD_Sweep()
time.sleep(1)
VDD_INIT()
PMOS_INIT()
VDD_On()
PMOS_On()
time.sleep(1)

NMOS_Sweep()
time.sleep(1)
NMOS_INIT()
NMOS_On()
time.sleep(1)

PMOS_Sweep()
time.sleep(1)
PMOS_INIT()
PMOS_On()   
time.sleep(1)


###############################################################
##################### Shut Down Procedure #####################
###############################################################
SM_PMOS.write(":OUTP OFF")
SM_NMOS.write(":OUTP OFF")
SM_VDD.write(":OUTP OFF")
                               
CHAMBER.write("SET=30")                             # Sets the temperature back to room temp once test is complete                      
while (CHAMBER.query("STATUS?")[3] == 'N'):         # Checks if chamber has arrived at room temp
    time.sleep(15)
CHAMBER.write("STOP")                               # Once chamber is at room temp, chamber stops operation

In [ ]:
SM_PMOS.write("OUTP:STAT 0")                                 # Turn off the VOUT source meter
SM_NMOS.write("OUTP:STAT 0") 
SM_VDD.write("OUTP:STAT 0")

In [ ]:
#CHAMBER.write("SET=25")
CHAMBER.write("STOP")